<a href="https://colab.research.google.com/github/romitbarua/MultiModalDeepFake/blob/main/TestingNotebooks/PhonemeVsDlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount the google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install moviepy
!pip3 install imageio==2.4.1
!pip install imageio-ffmpeg
!pip install textgrid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 14.7 MB/s 
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303886 sha256=925c2eb334f88e3e5f68788dc00023bab0135f90c48f3a2480436441260730ea
  Stored in directory: /root/.cache/pip/wheels/be/7b/04/4d8d56f1d503e5c404f0de6018c0cfa592c71588a39b49e002
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.9 MB 1.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#import appropriate packages
import os
import cv2
import dlib
from google.colab.patches import cv2_imshow
import numpy as np 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import librosa
from scipy.io import wavfile
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
import itertools
from moviepy.editor import *
import textgrid

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2007040/45929032 bytes (4.4%)4390912/45929032 bytes (9.6%)7004160/45929032 bytes (15.2%)8511488/45929032 bytes (18.5%)10993664/45929032 bytes (23.9%)13115392/45929032 bytes (28.6%)14729216/45929032 bytes (32.1%)17326080/45929032 bytes (37.7%)19398656/45929032 bytes (42.2%)21651456/45929032 bytes (47.1%)24010752/45929032 bytes (52.3%)26574848/45929032 bytes (57.9%)295

In [4]:
SYM_PATH = '/content/drive/MyDrive/DeepFakeDetection'
%cd $SYM_PATH
%pip install -e .

/content/drive/MyDrive/DeepFakeDetection
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/DeepFakeDetection
  Running setup.py develop for DeepFake


In [5]:
#import custom packages
from packages.DlibManager import DlibManager

predictor_path = '/content/drive/MyDrive/DeepFakeDetection/model/shape_predictor_68_face_landmarks.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)
     

In [6]:
#load the metadata
id = 'id01610'

metadata = pd.read_csv('/content/drive/MyDrive/DeepFakeDetection/data/full_metadata.csv')
metadata = metadata[(metadata['valid_dlib']==True) & (metadata['phoneme_path'].isna() == False)]

In [7]:
real_videos = metadata[metadata['method'] == 'real'].sample(frac=1, replace=False).reset_index(drop=True)

train_perc = 0.8

train_real = real_videos[0:int(real_videos.shape[0]*train_perc)]
test_real = real_videos[int(real_videos.shape[0]*train_perc):]

In [9]:
def distance(x1, x2):
  return np.sqrt(np.sum((x1 - x2)**2))

In [10]:
def normalize(ar, mean=None, std=None):
  if isinstance(mean, type(None)):
    mean = np.mean(ar)
  if isinstance(std, type(None)):
    std = np.std(ar)
    
  return (ar - mean)/std

In [68]:
def generate_features(dlib_video, phonemes, phoneme_time_adj, dlib_frame_ext, full_path):

  top_lip_idx = 52
  bottom_lip_idx = 58
  left_lip_idx = 49
  right_lip_idx = 55
  nose_idx = 34
  chin_idx = 9

  features = []

  duration = phonemes[-1].maxTime

  dlib_frames = dlib_video.full_frames
  landmarks = dlib_video.landmarks

  second_per_frame = duration/len(dlib_frames)
  dlib_frame_times = np.arange(0, len(dlib_frames), 1)*second_per_frame


  full_video_vert_lip_distance = np.array([distance(landmark[top_lip_idx], landmark[bottom_lip_idx]) for landmark in landmarks])
  full_video_hor_lip_distance = np.array([distance(landmark[left_lip_idx], landmark[right_lip_idx]) for landmark in landmarks])
  full_video_nose_chin_distance = np.array([distance(landmark[nose_idx], landmark[chin_idx]) for landmark in landmarks])
  full_video_mouth_ratio = full_video_vert_lip_distance/full_video_hor_lip_distance

  full_video_vert_mean = np.mean(full_video_vert_lip_distance)
  full_video_vert_std = np.std(full_video_vert_lip_distance)
  full_video_hor_mean = np.mean(full_video_hor_lip_distance)
  full_video_hor_st = np.std(full_video_hor_lip_distance)
  full_video_nose_chin_mean = np.mean(full_video_nose_chin_distance)
  full_video_nose_chin_std = np.std(full_video_nose_chin_distance)
  full_video_mouth_ratio_mean = np.mean(full_video_mouth_ratio)
  full_video_mouth_ratio_std = np.std(full_video_mouth_ratio)

  for phoneme in phonemes:

    start_time, end_time = phoneme.minTime-phoneme_time_adj, phoneme.maxTime+phoneme_time_adj

    start_frame_idx = max(abs(dlib_frame_times - start_time).argmin() - dlib_frame_ext,0)
    end_frame_idx = min(abs(dlib_frame_times - end_time).argmin() + dlib_frame_ext, len(dlib_frames))

    phoneme_landmark = landmarks[start_frame_idx:end_frame_idx]

    vert_lip_distance = np.array([distance(landmark[top_lip_idx], landmark[bottom_lip_idx]) for landmark in phoneme_landmark])
    hor_lip_distance = np.array([distance(landmark[left_lip_idx], landmark[right_lip_idx]) for landmark in phoneme_landmark])
    nose_chin_distance = np.array([distance(landmark[nose_idx], landmark[chin_idx]) for landmark in phoneme_landmark])
    mouth_ratio = vert_lip_distance/hor_lip_distance

    vert_lip_distance = normalize(vert_lip_distance, full_video_vert_mean, full_video_vert_std)
    hor_lip_distance = normalize(hor_lip_distance, full_video_hor_mean, full_video_hor_st)
    nose_chin_distance = normalize(nose_chin_distance, full_video_nose_chin_mean, full_video_nose_chin_std)
    mouth_ratio = normalize(mouth_ratio, full_video_mouth_ratio_mean, full_video_mouth_ratio_std)

    vert_lip_distance_max = vert_lip_distance.max()
    vert_lip_distance_min = vert_lip_distance.min()
    vert_lip_distance_diff = vert_lip_distance_max - vert_lip_distance_min
    vert_lip_distance_std = vert_lip_distance.std()

    hor_lip_distance_max = hor_lip_distance.max()
    hor_lip_distance_min = hor_lip_distance.min()
    hor_lip_distance_diff = hor_lip_distance_max - hor_lip_distance_min
    hor_lip_distance_std = hor_lip_distance.std()

    nose_chin_distance_max = nose_chin_distance.max()
    nose_chin_distance_min = nose_chin_distance.min()
    nose_chin_distance_diff = nose_chin_distance_max - nose_chin_distance_min
    nose_chin_distance_std = nose_chin_distance.std()

    mouth_ratio_max = mouth_ratio.max()
    mouth_ratio_min = mouth_ratio.min()
    mouth_ratio_diff = mouth_ratio_max - mouth_ratio_min
    mouth_ratio_std = mouth_ratio.std()

    features.append([full_path, phoneme.mark, len(phoneme_landmark), start_time, end_time, start_frame_idx, end_frame_idx,
                     vert_lip_distance_max, vert_lip_distance_min, vert_lip_distance_diff, vert_lip_distance_std,
                     hor_lip_distance_max, hor_lip_distance_min, hor_lip_distance_diff, hor_lip_distance_std, 
                     nose_chin_distance_max, nose_chin_distance_min, nose_chin_distance_diff, nose_chin_distance_std,
                     mouth_ratio_max, mouth_ratio_min, mouth_ratio_diff, mouth_ratio_std])
    
  return features


  






In [75]:
#1 for each video, extract the phoneme
#2 for each phoneme, using moviepy, get the frames associated with the phoneme
#3 get the landmarks associated with each frame
#4 return list of phoneme & landmarks
def extract(data, phoneme_time_adj, dlib_frame_ext):

  full_results = []

  for index, row in data.iterrows():

    video_results = []

    print(f'\r Running Video {index+1} of {data.shape[0]}', end='')


    phonemes = textgrid.TextGrid.fromFile(row['phoneme_path'])[1]

    video = cv2.VideoCapture(row['full_path'])
    dlib_video = DlibManager(detector=detector, predictor=predictor, video=video)

    full_results.extend(generate_features(dlib_video, phonemes, phoneme_time_adj, dlib_frame_ext, row['full_path']))

  return pd.DataFrame(full_results, columns=['full_path', 'phoneme', 'num_frames', 'phoneme_start', 'phoneme_end', 'frame_start', 'frame_end',
                                            'vert_lip_max', 'vert_lip_min', 'vert_lip_diff', 'vert_lip_std',
                                            'hor_lip_max', 'hor_lip_min', 'hor_lip_diff', 'hor_lip_std',
                                            'nose_chin_max', 'nose_chin_min', 'nose_chin_diff', 'nose_chin_std',
                                            'mouth_ratio_max', 'mouth_ratio_min', 'mouth_ratio_diff', 'mouth_ratio_std'])

    



In [76]:
train_df = extract(train_real, 0, 1)
test__real_df = extract(test_real, 0, 1)

 Running Video 425 of 85

In [79]:
train_df.columns

Index(['full_path', 'phoneme', 'num_frames', 'phoneme_start', 'phoneme_end',
       'frame_start', 'frame_end', 'vert_lip_max', 'vert_lip_min',
       'vert_lip_diff', 'vert_lip_std', 'hor_lip_max', 'hor_lip_min',
       'hor_lip_diff', 'hor_lip_std', 'nose_chin_max', 'nose_chin_min',
       'nose_chin_diff', 'nose_chin_std', 'mouth_ratio_max', 'mouth_ratio_min',
       'mouth_ratio_diff', 'mouth_ratio_std'],
      dtype='object')

In [21]:
#get the dlib landmarks

real_videos = []


for path in fakeAV_real[:2]:

  video = cv2.VideoCapture(path)
  dlib_video = DlibManager(detector=detector, predictor=predictor, video=video)

  phoneme_path = path.replace('/FakeAVCeleb_v1.2/', '/FakeAVCeleb_v1.2/phonetic_alignment_files/').replace('.mp4', '.TextGrid')
  phoneme = textgrid.TextGrid.fromFile(phoneme_path)
  print(phoneme)




  

NameError: ignored